## 4) Write your own codes for Lane Detection

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [4]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    
    return cv2.bitwise_and(img, mask)

In [5]:
def full_pipeline(img):
    height, width = img.shape[:2]
    
    # 흑백이미지 변환
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Blur
    blur_img = cv2.GaussianBlur(gray_img, (3, 3), 0)
    
    # Canny edge
    canny_img = cv2.Canny(blur_img, 70, 210)
    
    # 사다리꼴
    vertices = np.array([[(50, height),
                          (width/2-45, height/2+60), 
                          (width/2+45, height/2+60), 
                          (width-50,height)]], dtype='int')
    # ROI 설정
    ROI_img = region_of_interest(canny_img, vertices)
    
    # 허프 변환
    lines = cv2.HoughLinesP(ROI_img, 1, 1 * np.pi/180, 30, np.array([]), minLineLength= 10, maxLineGap=20)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), [0, 0, 255], 2)
    
    hough_img =line_img
    
    # 원본 이미지에 검출된 선 overlap
    result = cv2.addWeighted(hough_img, 1, img, 1, 0)
    print(hough_img.size)
    print(img.size)
    return result

In [46]:
listOfFiles = ['challenge.jpg','solidWhiteRight.jpg', 'solidYellowLeft.jpg']

for i, file in enumerate(listOfFiles):
    img = cv2.imread(file)
    
    result = full_pipeline(img)
    cv2.imshow('lane detection', result)
    cv2.waitKey(0) 
    cv2.imwrite("result.jpg", result)
cv2.destroyAllWindows()

1555200
1555200
1555200
1555200
1555200
1555200


# ※ Further improvements

In [6]:
def improved_pipeline(img):
    #1) 기울기 이용
    #2) 차선당 하나의 직선으로 표현
    
    height, width = img.shape[:2]
    
    # 흑백이미지 변환
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Blur
    blur_img = cv2.GaussianBlur(gray_img, (3, 3), 0)
    
    # Canny edge
    canny_img = cv2.Canny(blur_img, 70, 210)
    
    # 사다리꼴
    vertices = np.array([[(50, height),
                          (width/2-45, height/2+60), 
                          (width/2+45, height/2+60), 
                          (width-50,height)]], dtype='int')
    # ROI 설정
    ROI_img = region_of_interest(canny_img, vertices)
    
    # 허프 변환
    lines = cv2.HoughLinesP(ROI_img, 1, 1 * np.pi/180,  30, np.array([]), minLineLength= 10, maxLineGap=20)
    #line_img = np.zeros(img.shape, dtype=np.uint8)
    lines = np.squeeze(lines)
    
    # 기울기 구하기
    slope_degree = (np.arctan2(lines[:,1] - lines[:,3], lines[:,0] - lines[:,2]) * 180) / np.pi

    # 수평 기울기 제한
    lines = lines[np.abs(slope_degree)<160]
    slope_degree = slope_degree[np.abs(slope_degree)<160]
   
    # 수직 기울기 제한
    lines = lines[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
    
    # 필터링된 직선 버리기
    L_lines, R_lines = lines[(slope_degree>0),:], lines[(slope_degree<0),:]
    temp = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]
    
    for line in L_lines:
        for x1,y1,x2,y2 in line:
            cv2.line(temp, (x1, y1), (x2, y2), [0, 0, 255], 2)
    
    for line in R_lines:
        for x1,y1,x2,y2 in line:
            cv2.line(temp, (x1, y1), (x2, y2), [0, 0, 255], 2)
    
    result = cv2.addWeighted(temp, 1, img, 1, 0)

    return result

In [7]:
def get_fitline(img, f_lines): # 대표선 구하기   
    
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0]*2,2)
    
    rows,cols = img.shape[:2]
    
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx + x) , img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx + x) , int(img.shape[0]/2+100)
    
    result = [x1,y1,x2,y2]
    
    return result

In [8]:
def improved2_pipeline(img):
    #1) 기울기 이용
    #2) 차선당 하나의 직선으로 표현
    
    height, width = img.shape[:2]
    
    # 흑백이미지 변환
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Blur
    blur_img = cv2.GaussianBlur(gray_img, (3, 3), 0)
    
    # Canny edge
    canny_img = cv2.Canny(blur_img, 70, 210)
    
    # 사다리꼴
    vertices = np.array([[(50, height),
                          (width/2-45, height/2+6), 
                          (width/2+45, height/2+80), 
                          (width-50,height)]], dtype='int')
    # ROI 설정
    ROI_img = region_of_interest(canny_img, vertices)
    
    # 허프 변환
    lines = cv2.HoughLinesP(ROI_img, 1, 1 * np.pi/180,  30, np.array([]), minLineLength= 10, maxLineGap=20)
    #line_img = np.zeros(img.shape, dtype=np.uint8)
    lines = np.squeeze(lines)
    
    # 기울기 구하기
    slope_degree = (np.arctan2(lines[:,1] - lines[:,3], lines[:,0] - lines[:,2]) * 180) / np.pi

    # 수평 기울기 제한
    lines = lines[np.abs(slope_degree)<160]
    slope_degree = slope_degree[np.abs(slope_degree)<160]
   
    # 수직 기울기 제한
    lines = lines[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
    
    # 필터링된 직선 버리기
    L_lines, R_lines = lines[(slope_degree>0),:], lines[(slope_degree<0),:]
    temp = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]
    
    #직선 그리기
    left_fit_line = get_fitline(img,L_lines)
    right_fit_line = get_fitline(img,R_lines)
            
    cv2.line(temp, (left_fit_line[0], left_fit_line[1]), (left_fit_line[2], left_fit_line[3]), [0,0,255], 15)
    cv2.line(temp, (right_fit_line[0], right_fit_line[1]), (right_fit_line[2], right_fit_line[3]), [0,0,255], 15)
    
    result = cv2.addWeighted(temp, 1, img, 1, 0)

    return result

In [9]:
listOfFiles = ['solidWhiteRight.jpg', 'solidYellowLeft.jpg', 'challenge.jpg']

for i, file in enumerate(listOfFiles):
    img = cv2.imread(file)
    
    result = improved2_pipeline(img)
    cv2.imshow('lane detection', result)
    cv2.waitKey(0) 
    cv2.imwrite("improved_result.jpg", result)
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 4 into shape (8,2)